# Import the necessary libraries 

In [ ]:
import rasterio
import rasterio.plot
import pyproj
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import csv
import pandas as pd
import matplotlib.animation as animation
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors


## Configure the variables

In [ ]:
rasters_path = './rasters/vegetation-maps/'

filename_prefix = 'vegetation_mt_'

filename_suffix = '.tif'

years = ['2002', '2006', '2010',  '2014',  '2018',  '2022']

indexes = ['extreme','severe', 'moderate', 'light', 'no']


## Compute the gravity centers for all the considered years 

In [ ]:

results_longitude = np.zeros((len(indexes),len(years)))
results_latitude = np.zeros((len(indexes), len(years)))
results_coordinates_x = np.zeros((len(indexes),len(years)))
results_coordinates_y = np.zeros((len(indexes),len(years)))
results_area = np.zeros((len(indexes),len(years)))
for index,year in enumerate(years):
    file_name = rasters_path + filename_prefix + year + filename_suffix
    src = rasterio.open(file_name)
    array = src.read(1)
    for i in range(5):
        my_class = np.argwhere(array == i)
        my_mean = my_class.mean(axis=0)
        results_longitude[i, index] = src.xy(int(my_mean[0]), int(my_mean[1]))[0]
        results_latitude[i, index] = src.xy(int(my_mean[0]), int(my_mean[1]))[1]
        results_coordinates_x[i, index] = int(my_mean[0])
        results_coordinates_y[i, index] = int(my_mean[1])
        results_area[i, index] = int(my_class.shape[0]*0.03*0.03)
    src.close()
    print(year)

## Compute the areas of the different classes

In [ ]:
df_area = pd.DataFrame(results_area, index=indexes, columns=years)
sum = df_area.sum()
sum.name = 'Sum'
df_area = df_area.append(sum.transpose())

## Display a table with the area in km<sup>2</sup> of each class for all the years

In [ ]:
display(df_area)

In [ ]:
df_coord_x = pd.DataFrame(results_coordinates_x, index=indexes, columns=years)
df_coord_y = pd.DataFrame(results_coordinates_y, index=indexes, columns=years)
df_long = pd.DataFrame(results_longitude, index=indexes, columns=years)
df_lat = pd.DataFrame(results_latitude, index=indexes, columns=years)

### Gravity center longitude for the different classes

In [ ]:
display(df_long)

### Gravity center latitude for the different classes

In [ ]:
display(df_lat)

## Store the dataframes into a csv file to improve the computation

In [ ]:
df_long.to_csv("gc_long.csv" , index=True, header=True)
df_lat.to_csv("gc_lat.csv" , index=True, header=True)
df_area.to_csv("areas.csv", index=True, header=True)
df_coord_x.to_csv("coor_x.csv", index=True, header=True)
df_coord_y.to_csv("coor_y.csv", index=True, header=True)



## Read the csv files directly.

In [ ]:
df_long = pd.read_csv("gc_long.csv")
df_lat = pd.read_csv("gc_lat.csv")
df_area = pd.read_csv("areas.csv")
df_coord_x = pd.read_csv("coor_x.csv")
df_coord_y = pd.read_csv("coor_y.csv")
    
results_coordinates_x = df_coord_x.to_numpy()
results_coordinates_x = np.delete(results_coordinates_x, 0, 1)
results_coordinates_y = df_coord_y.to_numpy()
results_coordinates_y = np.delete(results_coordinates_y, 0, 1)


### Read a raster in order to use to display the gravity centers

In [ ]:
file_name = rasters_path + filename_prefix + years[0] + filename_suffix
src = rasterio.open(file_name)
array = src.read(1)
array[array != 255] = 0

## Display the Gravity Centers for all the classes for all the considered years in the considered raster

In [ ]:
from IPython.display import display, Markdown

cmap = ListedColormap(["lightgrey", "lightgrey", "lightgrey", "lightgrey", "lightgrey", "white"])
norm = colors.BoundaryNorm([0, 1, 2, 3, 4, 255], 6)
colors_gc = ["yellow", "#FFA500", "brown", "#90EE90", "green"]
markers_gc= ['o', '^', '*', 's', '+']




for idx, year in enumerate(years):
        
    display(Markdown(f"---"))
    display(Markdown(f"## Gravity Centers position of the 5 classes for the year " + year))
    
    plt.imshow(array, cmap=cmap, norm=norm)
    for i in range(results_coordinates_x.shape[0]):
        plt.scatter(results_coordinates_y[i, idx], results_coordinates_x[i, idx] , s=80, c=colors_gc[i], marker=markers_gc[i], label=indexes[i])
    img_title = "gravity centers of classes for the year " + year
    plt.title(img_title)
    legend = plt.legend(loc='lower right')
    legend.set_alpha(None)
    plt.axis('off')

    plt.show()
    src.close()
    

## Gravity centers localization for all the classes through the years


In [ ]:
import matplotlib.patches as mpatch
import matplotlib.path as mpath

cmap = ListedColormap(["lightgrey", "lightgrey", "lightgrey", "lightgrey", "lightgrey", "white"])
norm = colors.BoundaryNorm([0, 1, 2, 3, 4, 255], 6)
colors_gc = ['#8B0000', 'red', '#FF4500', '#00008B', '#006400']
markers_gc= ['*', 'o', 'v', '>', 'P', 's']
colors2 = ["yellow", "#FFA500", "brown", "#90EE90", "green"]

fig, ax = plt.subplots()

plt.imshow(array, cmap=cmap, norm=norm)




for i in range(len(indexes)):
    for idx in range(0, len(years)):        
        if idx == 2:
            plt.scatter(df_coord_y.iloc[i, idx+1], df_coord_x.iloc[i, idx+1], alpha=0.7, s=30,color=colors2[i], marker=markers_gc[idx//2], zorder=2+i, label=indexes[i], edgecolors='k', lw=0.25)            
        else:
            plt.scatter(df_coord_y.iloc[i, idx+1], df_coord_x.iloc[i, idx+1], alpha=0.7, s=30,color=colors2[i], marker=markers_gc[idx//2], zorder=2+i, edgecolors='k', lw=0.5)


legend = plt.legend(loc='lower center', title="Classes")
ax.add_artist(legend)


h = [plt.plot([],[], color="k", marker=markers_gc[i], ms=8, mec='k', ls="", markeredgewidth=0.25)[0] for i in range(len(years))]
plt.legend(handles=h, labels=years,loc='lower left', title="Years")

#legend.set_alpha(None)
plt.axis('off')
plt.savefig('gravity_centers_last.png', dpi=600)
plt.show()

In [ ]:
for i in range(len(indexes)):
    display(Markdown(f"---"))
    display(Markdown(f"## Gravity Centers movement through the years for the {indexes[i]} class " ))
    
    Xgc = []
    Ygc = []
    for idx in range(0, len(years)):
        Xgc.append(df_coord_y.iloc[i, idx+1])
        Ygc.append(df_coord_x.iloc[i, idx+1])
        plt.scatter(df_coord_y.iloc[i, idx+1], df_coord_x.iloc[i, idx+1], alpha=1, s=250,color=colors2[i], marker=markers_gc[idx], zorder=20-idx, edgecolors='k')
        #if idx in [0, len(years_2)-1]:
        #    plt.text(df_coord_y.iloc[i, idx+1], df_coord_x.iloc[i, idx+1]-150, years_2[idx], horizontalalignment='center', size='small', color='black')

        if idx != len(years) - 1:
            plt.plot([df_coord_y.iloc[i, idx+1], df_coord_y.iloc[i, idx+2]], [df_coord_x.iloc[i, idx+1], df_coord_x.iloc[i, idx+2]], alpha=1, linewidth=1,color='black', zorder=1)
            
    for idx in range(len(years) - 1):
        plt.annotate("", xy=((Xgc[idx]+Xgc[idx+1])/2, (Ygc[idx]+Ygc[idx+1])/2), xytext=(Xgc[idx], Ygc[idx]), 
            arrowprops=dict(arrowstyle="->", mutation_scale=25,  color='0', linewidth=1))
    print(i, max(Xgc), min(Xgc), max(Xgc) - min(Xgc))
    print(i, max(Ygc), min(Ygc), max(Ygc) - min(Ygc))

    plt.gca().invert_yaxis()
    
    plt.gca().xaxis.set_tick_params(labelbottom=False)
    plt.gca().yaxis.set_tick_params(labelleft=False)
    
    plt.gca().set_xticks([])
    plt.gca().set_yticks([])
    plt.savefig('gravity_centers_'+ indexes[i]+'.png', dpi=600)
    plt.show()

In [ ]:
import math
for idx in range(0, len(years) - 1 ):


    
    for idx2 in range(idx+1, len(years)):
        display(Markdown(f"---"))
        display(Markdown(f"## The Movement of all the gravity centers from {years[idx]} to {years[idx2]}" ))
        plt.scatter(0, 0, alpha=1, s=200,color='white', marker=markers_gc[idx], zorder=10, edgecolors='0')
        for i in range(len(indexes)):
            y_dif = df_coord_y.iloc[i, idx2+1] - df_coord_y.iloc[i, idx+1]
            x_dif = df_coord_x.iloc[i, idx2+1] - df_coord_x.iloc[i, idx+1]
            plt.scatter(y_dif, x_dif, alpha=1, s=200,color=colors2[i], marker=markers_gc[idx+1], zorder=0, edgecolors=colors2[i])
            plt.annotate("", xy=(y_dif, x_dif), xytext=(0, 0), 
                arrowprops=dict(arrowstyle="->", mutation_scale=25,  color=colors2[i], linewidth=3))
            print(f"The gravity center of the {indexes[i]} class moved {round(abs(y_dif)*0.03, 2)} to the {'north' if y_dif > 0 else 'south'} and {round(abs(x_dif)*0.03, 2)} to the {'east' if x_dif < 0 else 'west'}")
        #mean_X = (max(Xgc) + min(Xgc))/2
        #mean_Y = (max(Ygc) + min(Ygc))/2
        #a = -2.56200316e-01
        #plt.axline((0,0),   slope=a, color='grey',linestyle='dashed')
        plt.gca().invert_yaxis()

        plt.gca().xaxis.set_tick_params(labelbottom=False)
        plt.gca().yaxis.set_tick_params(labelleft=False)

        plt.gca().set_xticks([])
        plt.gca().set_yticks([])
        
        plt.savefig(f'gravity_centers_movement_from_{years[idx]}_to_{years[idx+1]}.png', dpi=600)
        plt.show()